Build an ElectronicStructureProblem

In [2]:
'''PySCFDriver is a class that allows to build a molecluar object in PySCF. We use it to compute the one body and two body integrals.'''
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g", #slater-type orbitals approximated with 3 gaussians
    charge=0, #int chrge of the molecule
    spin=0, #2S where S spin of the molecule
    unit=DistanceUnit.ANGSTROM,
)

In [3]:
# Problem is a ElectronicStructureProblem object
problem = driver.run()
print(problem)

/home/codespace/.python/current/lib/python3.10/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [4]:
# The ElectonicStructureProblem object contains the hamiltonian of the molecule, and electronic integrals which the one and two body integrals
hamiltonian = problem.hamiltonian
coefficients = hamiltonian.electronic_integrals #one and two body integrals are 2d and 4d tensors respectively
print(coefficients.alpha)

Polynomial Tensor
 "+-":
[[-1.25633907e+00 -1.37083854e-17]
 [-6.07732712e-17 -4.71896007e-01]]
 "++--":
[[[[6.75710155e-01 1.69253442e-16]
   [1.56722377e-16 1.80931200e-01]]

  [[4.84650299e-17 1.80931200e-01]
   [6.64581730e-01 3.79897400e-16]]]


 [[[1.01440795e-16 6.64581730e-01]
   [1.80931200e-01 4.71502663e-17]]

  [[1.80931200e-01 3.78920172e-16]
   [6.59828421e-17 6.98573723e-01]]]]


In [5]:
#The hamiltonian object contains the second quantized operator (the two summands of the hamiltonian). 
#This is the electronic part of the hamiltonian (nuclear repulsion energy will be added later as a constant, is available as hamiltonian.nuclear_repulsion_energy)
fermionic_op = hamiltonian.second_q_op()
print(fermionic_op)

Fermionic Operator
number spin orbitals=4, number terms=36
  0.33785507740175813 * ( +_0 +_0 -_0 -_0 )
+ 0.09046559989211568 * ( +_0 +_0 -_1 -_1 )
+ 0.09046559989211564 * ( +_0 +_1 -_0 -_1 )
+ 0.33229086512764827 * ( +_0 +_1 -_1 -_0 )
+ 0.33785507740175813 * ( +_0 +_2 -_2 -_0 )
+ 0.09046559989211568 * ( +_0 +_2 -_3 -_1 )
+ 0.09046559989211564 * ( +_0 +_3 -_2 -_1 )
+ 0.33229086512764827 * ( +_0 +_3 -_3 -_0 )
+ 0.3322908651276482 * ( +_1 +_0 -_0 -_1 )
+ 0.09046559989211574 * ( +_1 +_0 -_1 -_0 )
+ 0.09046559989211565 * ( +_1 +_1 -_0 -_0 )
+ 0.3492868613660089 * ( +_1 +_1 -_1 -_1 )
+ 0.3322908651276482 * ( +_1 +_2 -_2 -_1 )
+ 0.09046559989211574 * ( +_1 +_2 -_3 -_0 )
+ 0.09046559989211565 * ( +_1 +_3 -_2 -_0 )
+ 0.3492868613660089 * ( +_1 +_3 -_3 -_1 )
+ 0.33785507740175813 * ( +_2 +_0 -_0 -_2 )
+ 0.09046559989211568 * ( +_2 +_0 -_1 -_3 )
+ 0.09046559989211564 * ( +_2 +_1 -_0 -_3 )
+ 0.33229086512764827 * ( +_2 +_1 -_1 -_2 )
+ 0.33785507740175813 * ( +_2 +_2 -_2 -_2 )
+ 0.09046559989211568

Mapping the problem to qubit space

In [6]:
'''Jordan-Wigner transformation maps the fermionic operators to unitary operators acting on qubits (https://arxiv.org/abs/2110.12792).
Doing so, we can map the n-body fermionic operator to a circuit of n-qubit gates.
'''
'''EDIT: bug in qiskit_nature.second_q.transforms.qubit_mapper.py called by JordanWignerMapper. Calls to_do method on FermionicOp object, 
which is not implemented, as the note says. Another mapper should be used, e.g. ParityMapper'''

from qiskit_nature.second_q.mappers import ParityMapper
mapper = ParityMapper()

In [7]:
qubit_jw_op = mapper.map(fermionic_op)
print(qubit_jw_op)

-0.8105479805373275 * IIII
- 0.22575349222402394 * IIZZ
+ 0.17218393261915554 * IIIZ
+ 0.12091263261776629 * IIZI
+ 0.1721839326191556 * IZZI
+ 0.16892753870087907 * IZZZ
+ 0.04523279994605784 * ZXIX
- 0.04523279994605784 * IXZX
- 0.04523279994605784 * ZXZX
+ 0.04523279994605784 * IXIX
- 0.22575349222402388 * ZZII
+ 0.16614543256382414 * ZZIZ
+ 0.16614543256382414 * IZIZ
+ 0.17464343068300445 * ZZZZ
+ 0.12091263261776629 * ZIZI


/home/codespace/.python/current/lib/python3.10/site-packages/qiskit_nature/deprecation.py:297: PauliSumOpDeprecationWarning: PauliSumOp is deprecated as of version 0.6.0 and support for them will be removed no sooner than 3 months after the release. Instead, use SparsePauliOp. You can switch to SparsePauliOp immediately, by setting `qiskit_nature.settings.use_pauli_sum_op` to `False`.
  return func(*args, **kwargs)


Classical Solver

In [8]:
'''ow we need to define a solver, an algorithm that will find the ground state of the hamiltonian. 
One example is the NumPyEigensolver, which uses the NumPy sparse eigenvalue solver to compute the eigenvalues. This is a classical algorithm'''
from qiskit.algorithms.eigensolvers import NumPyEigensolver

numpy_solver = NumPyEigensolver(k=1)
numpy_result = numpy_solver.compute_eigenvalues(qubit_jw_op)
print(numpy_result)

{   'aux_operators_evaluated': None,
    'eigenstates': [   Statevector([ 2.89134920e-17+1.19576597e-19j,
             -1.03085586e-16-1.11995585e-16j,
              4.31376602e-18-1.03035986e-17j,
             -9.81323962e-01+1.56725926e-01j,
              3.97188390e-17-4.77700973e-17j,
             -9.42341599e-17-1.44958222e-16j,
              1.10140120e-01-1.75903299e-02j,
             -4.20482361e-17+2.17098702e-18j,
              1.76614048e-16+3.26584074e-17j,
             -1.14699657e-17-2.55538386e-16j,
              3.64766425e-16-2.58409077e-16j,
             -5.73658153e-18-2.44861468e-17j,
             -6.20497151e-17+6.09058146e-17j,
             -7.02625786e-19-7.30695595e-18j,
              1.99212996e-16-3.25288538e-16j,
             -4.81491726e-17+4.38892757e-17j],
            dims=(2, 2, 2, 2))],
    'eigenvalues': array([-1.85727503])}
